In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#==============================
# 1) Install & Imports
#==============================
!pip -q install torch torchvision pandas tqdm

import torch, torch.nn as nn, torch.nn.functional as F
from torchvision import transforms
from torchvision.models import resnet18
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pandas as pd
from tqdm.auto import tqdm
from pathlib import Path


In [3]:
#==============================
# 2) Distill DataLoader
#==============================
# --- パス設定 (必要に応じて書き換え) ---
CSV_TRAIN = Path("/content/drive/MyDrive/AffectNet/train_pairs.csv")
IMG_SIZE  = 96
BATCH_D   = 128
NUM_W     = 2

class DistillDataset(Dataset):
    def __init__(self, csv_file):
        df = pd.read_csv(csv_file)
        self.paths = df["image_path"].tolist()
        self.au    = df["au_vector"].apply(eval).tolist()   # 35 次元 list
        self.tf = transforms.Compose([
            transforms.Resize((IMG_SIZE, IMG_SIZE)),
            transforms.ToTensor(),
            transforms.Normalize([0.5]*3, [0.5]*3),
        ])
    def __len__(self): return len(self.paths)
    def __getitem__(self, idx):
        img = Image.open(self.paths[idx]).convert("RGB")
        x   = self.tf(img)
        y   = torch.tensor(self.au[idx], dtype=torch.float32)  # shape (35,)
        return x, y

distill_dl = DataLoader(
    DistillDataset(CSV_TRAIN),
    batch_size=BATCH_D, shuffle=True,
    num_workers=NUM_W, pin_memory=True)

print(f"✅ Distill dataset: {len(distill_dl.dataset)} images")


✅ Distill dataset: 11183 images


In [4]:
#==============================
# 3) Train ResNet18 as AU detector
#==============================
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
EPOCHS = 10
LR     = 1e-3

# --- モデル定義 ---
net = resnet18(weights=None)
net.fc = nn.Sequential(nn.Linear(512, 35), nn.Sigmoid())  # 出力 35 dim
net = net.to(DEVICE)

opt = torch.optim.AdamW(net.parameters(), lr=LR, weight_decay=1e-4)

for ep in range(1, EPOCHS+1):
    net.train()
    running = 0
    for x, y in tqdm(distill_dl, desc=f"Epoch {ep}/{EPOCHS}", leave=False):
        x, y = x.to(DEVICE), y.to(DEVICE)
        pred = net(x)
        loss = F.l1_loss(pred, y)
        opt.zero_grad(); loss.backward(); opt.step()
        running += loss.item()
    print(f"Epoch {ep}:  MAE {running/len(distill_dl):.4f}")

# --- 保存 ---
from pathlib import Path
CKPT = Path("/content/drive/MyDrive/AU_distill/resnet18_au.pth")
CKPT.parent.mkdir(parents=True, exist_ok=True)   # ★ フォルダを自動作成
torch.save(net.state_dict(), CKPT)
print("🎉 Saved distilled AU model to", CKPT)


Epoch 1/10:   0%|          | 0/88 [00:00<?, ?it/s]

Epoch 1:  MAE 0.2646


Epoch 2/10:   0%|          | 0/88 [00:00<?, ?it/s]

Epoch 2:  MAE 0.2545


Epoch 3/10:   0%|          | 0/88 [00:00<?, ?it/s]

...

Epoch 8:  MAE 0.2355


Epoch 9/10:   0%|          | 0/88 [00:00<?, ?it/s]

Epoch 9:  MAE 0.2323


Epoch 10/10:   0%|          | 0/88 [00:00<?, ?it/s]

Epoch 10:  MAE 0.2273
🎉 Saved distilled AU model to /content/drive/MyDrive/AU_distill/resnet18_au.pth
